In [1]:
# ------------------------------------------------------------
# 2023-03-18
# https://gradient.paperspace.com/
# Machine: Paid-RTX4000 (GPU) | 30 GiB RAM | 8 CPU | 8 GiB GPU
# ------------------------------------------------------------

In [2]:
# for loading the pre-trained models
from cryspnet.models import *
# for feature generation and plotting
from cryspnet.utils import *
# for accessing some pre-defined constant
from cryspnet.config import *

In [3]:
import pandas as pd
import numpy as np

# Load in the Pre-Trained Models 

These three API helps you to retrieved them
```Python
    load_Bravais_models(n_ensembler, which)
    load_Lattice_models()
    load_SpaceGroup_models()
```

**Note**: If the file cannot be found error showed, make sure your follow the instruction to download and extract those models to the correct place.

In [4]:
BE = load_Bravais_models(n_ensembler=5, which="whole")

In [5]:
LB = load_Lattice_models()

In [6]:
SGB = load_SpaceGroup_models()

# Download the Superhard Dataset

Matminer provide an easy to access API for fast dataset retrieval. Check out [this page](https://hackingmaterials.lbl.gov/matminer/dataset_summary.html) for datasets that are all available

In [7]:
from matminer.datasets import load_dataset

In [8]:
shard = load_dataset("brgoch_superhard_training")

Fetching brgoch_superhard_training.json.gz from https://ndownloader.figshare.com/files/13858931 to /notebooks/venv/lib/python3.9/site-packages/matminer/datasets/brgoch_superhard_training.json.gz


In [9]:
shard.head()

,formula,bulk_modulus,shear_modulus,composition,material_id,structure,brgoch_feats,suspect_value
0,AlPt3,225.230461,91.197748,"(Al, Pt)",mp-188,"[[0. 0. 0.] Al, [0. 1.96140395 1.96140395] Pt, [1.96140395 1.96140395 0. ] Pt, [1.96140395 0. 1.96140395] Pt]","{'atomic_number_feat_1': 123.5, 'atomic_number_feat_2': 221, 'atomic_number_feat_3': 13, 'atomic_number_feat_4': 78, 'atomic_weight_feat_1': 306.1107695, 'atomic_weight_feat_2': 558.258461, 'atomic_weight_feat_3': 26.981539, 'atomic_weight_feat_4': 195.08, 'period_number_feat_1': 10.5, 'period_number_feat_2': 15, 'period_number_feat_3': 3, 'period_number_feat_4': 6, 'group_number_feat_1': 21.5, 'group_number_feat_2': 17, 'group_number_feat_3': 10, 'group_number_feat_4': 13, 'family_number_feat_1': 8.5, 'family_number_feat_2': 7, 'family_number_feat_3': 4, 'family_number_feat_4': 5, 'Mendel...",False
1,Mn2Nb,232.696340,74.590157,"(Mn, Nb)",mp-12659,"[[-2.23765223e-08 1.42974191e+00 5.92614104e+00] Mn, [3.57977169 0.71487093 1.97538035] Mn, [1.23819283 0.71487093 1.97538035] Mn, [1.17078941 3.45760869 5.92614104] Mn, [0. 0. 0.] Mn, [0. 0. 3.9507607] Mn, [2.40898226 2.74273771 1.97538035] Mn, [-1.17078945 3.45760869 5.92614104] Mn, [-2.07564892e-08 2.78165309e+00 3.46044846e+00] Nb, [2.40898226 1.39082653 7.41120915] Nb, [2.40898226 1.39082653 4.44107293] Nb, [-2.07564892e-08 2.78165309e+00 4.90312238e-01] Nb]","{'atomic_number_feat_1': 45.5, 'atomic_number_feat_2': 9, 'atomic_number_feat_3': 25, 'atomic_number_feat_4': 41, 'atomic_weight_feat_1': 101.39124, 'atomic_weight_feat_2': 16.96972, 'atomic_weight_feat_3': 54.93805, 'atomic_weight_feat_4': 92.90638, 'period_number_feat_1': 6.5, 'period_number_feat_2': 3, 'period_number_feat_3': 4, 'period_number_feat_4': 5, 'group_number_feat_1': 9.5, 'group_number_feat_2': 9, 'group_number_feat_3': 5, 'group_number_feat_4': 7, 'family_number_feat_1': 6.0, 'family_number_feat_2': 4, 'family_number_feat_3': 4, 'family_number_feat_4': 4, 'Mendeleev_number_f...",False
2,HfO2,204.573433,98.564374,"(Hf, O)",mp-352,"[[2.24450185 3.85793022 4.83390736] O, [2.77883096 1.26035638 2.32337921] O, [ 2.79593023 1.33721745 -0.3395849 ] O, [2.26160112 3.93479129 2.17094325] O, [0.33243749 1.71503186 1.79217508] O, [4.72509386 4.3126057 0.03918327] O, [4.70799459 3.48011581 2.70214738] O, [0.31533822 0.88254197 4.45513919] O, [1.37618055 2.37584161 3.54068723] Hf, [3.64715226 4.97341544 3.61659934] Hf, [3.66425153 2.81930606 0.95363523] Hf, [1.39327982 0.22173223 0.87772312] Hf]","{'atomic_number_feat_1': 44.0, 'atomic_number_feat_2': 56, 'atomic_number_feat_3': 8, 'atomic_number_feat_4': 72, 'atomic_weight_feat_1': 105.2444, 'atomic_weight_feat_2': 146.4912, 'atomic_weight_feat_3': 15.9994, 'atomic_weight_feat_4': 178.49, 'period_number_feat_1': 5.0, 'period_number_feat_2': 2, 'period_number_feat_3': 2, 'period_number_feat_4': 6, 'group_number_feat_1': 18.0, 'group_number_feat_2': 28, 'group_number_feat_3': 4, 'group_number_feat_4': 16, 'family_number_feat_1': 9.0, 'family_number_feat_2': 10, 'family_number_feat_3': 4, 'family_number_feat_4': 7, 'Mendeleev_number_f...",False
3,Cu3Pt,159.312640,51.778816,"(Cu, Pt)",mp-12086,"[[0. 1.86144248 1.86144248] Cu, [1.86144248 1.86144248 0. ] Cu, [1.86144248 0. 1.86144248] Cu, [0. 0. 0.] Pt]","{'atomic_number_feat_1': 82.5, 'atomic_number_feat_2': 9, 'atomic_number_feat_3': 29, 'atomic_number_feat_4': 78, 'atomic_weight_feat_1': 192.859, 'atomic_weight_feat_2': 4.44200000000001, 'atomic_weight_feat_3': 63.546, 'atomic_weight_feat_4': 195.08, 'period_number_feat_1': 9.0, 'period_number_feat_2': 6, 'period_number_feat_3': 4, 'period_number_feat_4': 6, 'group_number_feat_1': 21.5, 'group_number_feat_2': 23, 'group_number_feat_3': 10, 'group_number_feat_4': 11, 'family_number_feat_1': 8.0, 'family_number_feat_2': 8, 'family_number_feat_3': 4, 'family_number_feat_4': 4, 'Mendeleev_nu...",False
4,Mg3Pt,69.637565,27.588765,"(Mg, Pt)",mp-18707,"[[0. 0. 2.73626461] Mg, [0. 0. 6.95125305] Mg, [-3.97440991e-08 4.58925330

# Generate Predictors

In [10]:
FG = FeatureGenerator()

In [11]:
predictors = FG.generate(shard[['formula']])

StrToComposition:   0%|          | 0/2574 [00:00<?, ?it/s]

MultipleFeaturizer:   0%|          | 0/2574 [00:00<?, ?it/s]

# Prediction-High Level API

Utilize the library is simple, every model has a **.predicts()** method that take a dataframe with ***predictors*** as input and output ***Bravais Lattices***, ***space group numbers*** or ***Lattice parameters***. 

## Bravais Lattice

Prediction from ***Bravais Lattice Models*** has two parts in the output: First one is the ***top-n highest probability in the prediction***. Second one is ***the classes that associated with those in the first one***. The top-n is controlled by the parameter **topn_bravais**. The first column of 

In [12]:
# first one is the probability of the top-N prediction
# second one is the predicted top-N Bravais Lattice
Bravais_probs, Bravais = BE.predicts(predictors, topn_bravais=2)

/notebooks/cryspnet/models.py:255: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  return preds[auxidxs, idxs], self.classes[idxs]


### Example-Bravais Lattice Prediction

The probability is order from left to right meaning the highest one are always on the left.

In [13]:
Bravais_probs[:2]

array([[0.8022752 , 0.03721721],
       [0.3946411 , 0.35970688]], dtype=float32)

Here is the corresponding Bravais Lattice

In [14]:
Bravais[:2]

array([['orthorhombic (P)', 'monoclinic (C)'],
       ['orthorhombic (C)', 'hexagonal (P)']], dtype=object)

## Lattice Parameter and Space Group

To use those two model, a column with the name **Bravais** must exists in the input predictors dataframe.
<br/>This information could be obtained from previous predictions or experimental data.

In [15]:
lattices = []
spacegroups = []
for i in range(0, 2):
    predictors['Bravais'] = Bravais[:, i]
    lattice = LB.predicts(predictors)
    lattices.append(lattice)
    
    spacegroup = SGB.predicts(predictors)
    spacegroups.append(spacegroup)

### Example-Lattice Parameter Prediction 
***Lattice Parameter Model*** would output a ***dataframe*** contains columns: ***a***, ***b***, ***c***, ***α***, ***β***,and ***γ***

In [16]:
lattices[0].head()

,a,b,c,alpha,beta,gamma,v
0,8.530208,9.565442,6.484684,90.0,90.0,90.0,529.119174
1,44.027000,23.800000,19.803800,90.0,90.0,90.0,20751.265282
2,6.845959,6.845959,6.845959,90.0,90.0,90.0,320.850679
3,8.325901,8.325901,8.325901,90.0,90.0,90.0,577.156664
4,20.097000,20.097000,20.097000,90.0,90.0,90.0,8116.965453


### Example-Space Group Prediction
***Space Group Model*** would output ***top-n probability*** and corresponding ***space group number***
in similar manner as Bravais Lattice Model

In [17]:
# top-N probability, top-N space group number
spacegroup_probs, spacegroup = spacegroups[0]

In [18]:
spacegroup_probs[:5]

array([[0.56252986, 0.36785939, 0.06961077],
       [1.        , 0.        , 0.        ],
       [0.73811179, 0.18276665, 0.0791216 ],
       [0.54353511, 0.27929729, 0.09348603],
       [1.        , 0.        , 0.        ]])

In [19]:
spacegroup[:5]

array([[ 62.,  74.,  61.],
       [ 64.,  63.,  36.],
       [225., 227., 216.],
       [220., 204., 229.],
       [225., 227., 216.]])

## Low Level API

### Load dataset to the model

All models contain **.load()** method to attach the dataset to the model. The model then would perform pre-processing on the input dataset. 

In [20]:
BE.load(predictors)
LB.load(predictors) # note we have "Bravais" in this dataframe already
SGB.load(predictors)

### Get the Raw Prediction
Sometimes, processing raw prediction is easy for further analysis. Make sure you load in dataset to the models before running lines in below.

The raw prediction is stored in the form of **torch tensor**. Prediction is stored in different format that is specified to each models. 

#### Bravais Lattice Model

For Bravais Lattice Model, raw prediction contains output from **N** (defined by user) models. i.e. the shape of the tensor is (N, dataset_size, 14)

In [21]:
preds = BE.get_preds()

In [22]:
preds.shape

torch.Size([5, 2574, 14])

The Bravais Lattice map to the predicted probability is stored in **.classes**

In [23]:
BE.classes

Index(['cubic (I)', 'cubic (F)', 'cubic (P)', 'hexagonal (P)',
       'rhombohedral (P)', 'tetragonal (I)', 'tetragonal (P)',
       'orthorhombic (I)', 'orthorhombic (F)', 'orthorhombic (C)',
       'orthorhombic (P)', 'monoclinic (C)', 'monoclinic (P)',
       'triclinic (P)'],
      dtype='object')

#### Space Group Model and Lattice Parameter Model

For Space Group Model and Lattice Parameter Model, the raw prediction is a dictionary containing prediction from different sub-models. The key of the dictionary shows which sub-models the prediction comes from.

In [24]:
preds = SGB.get_preds()

In [25]:
preds.keys()

dict_keys(['cubic (I)', 'cubic (F)', 'cubic (P)', 'hexagonal (P)', 'rhombohedral (P)', 'tetragonal (I)', 'tetragonal (P)', 'orthorhombic (I)', 'orthorhombic (F)', 'orthorhombic (C)', 'orthorhombic (P)', 'monoclinic (C)', 'monoclinic (P)', 'triclinic (P)'])

In [26]:
preds['cubic (F)'][:3,]

tensor([[0.0000, 1.0000, 0.0000],
        [0.0000, 0.0000, 1.0000],
        [0.0653, 0.3716, 0.5631]])

The space group number for each sub model prediction is little more complicated. All submodels is stored in the **.Ps** field. For each sub-model access classes is similar to Bravais Lattice one. Here is an example of getting the classes for cubic (F) sub-model

In [27]:
SGB.Ms['cubic (F)'].classes

array([216, 225, 227])

In [28]:
preds = LB.get_preds()

In [29]:
preds.keys()

dict_keys(['cubic (I)', 'cubic (F)', 'cubic (P)', 'hexagonal (P)', 'rhombohedral (P)', 'tetragonal (I)', 'tetragonal (P)', 'orthorhombic (I)', 'orthorhombic (F)', 'orthorhombic (C)', 'orthorhombic (P)', 'monoclinic (C)', 'monoclinic (P)', 'triclinic (P)'])

In [30]:
preds['triclinic (P)'][:3]

tensor([[ 2.4436, -1.6814,  1.7787, -2.1966, -1.9365, -1.8829],
        [-0.6659, -1.5837,  0.3591, -0.9489, -0.9671, -0.9137],
        [-1.3660, -1.6814, -1.5485, -2.1966, -1.9365, -1.8829]],
       dtype=torch.float64)

The **PRED_COLS** constant defined *config.py* tells you which lattice parameters the sub model is predicting.  

In [31]:
PRED_COLS

{'cubic (P)': ['a'],
 'cubic (F)': ['a'],
 'cubic (I)': ['a'],
 'monoclinic (P)': ['a', 'b', 'c', 'beta'],
 'monoclinic (C)': ['a', 'b', 'c', 'beta'],
 'hexagonal (P)': ['a', 'c'],
 'rhombohedral (P)': ['a', 'alpha'],
 'tetragonal (P)': ['a', 'c'],
 'tetragonal (I)': ['a', 'c'],
 'orthorhombic (P)': ['a', 'b', 'c'],
 'orthorhombic (C)': ['a', 'b', 'c'],
 'orthorhombic (F)': ['a', 'b', 'c'],
 'orthorhombic (I)': ['a', 'b', 'c'],
 'triclinic (P)': ['a', 'b', 'c', 'alpha', 'beta', 'gamma']}